# Pivot Table.

## Intro

Uma tabela pivot ou tabela dinâmica é uma ferramenta de resumo que geralmente está disponível em programas de planilha.

Cria medidas de resumo por uma ou mais chaves, usando essas chaves como rótulos de linha ou coluna.

O Pandas fornece dois métodos para a `pivot table` DataFrames.

- [`pandas.pivot_table()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html) 

- [`pandas.DataFrame.pivot_table()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot_table.html)

## Dataset

Nesta prática guiada, usaremos os dados da competição "titanic".

Este conjunto de dados também pode ser baixado em [mwaskom/seaborn-data/titanic.csv](https://github.com/mwaskom/seaborn-data/blob/master/titanic.csv).

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
data = sns.load_dataset('titanic')

C:\Users\UESLLEI\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Vejamos o tamanho e os primeiros registros do conjunto de dados:

In [2]:
data.shape

(891, 15)

In [4]:
data.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


## `pivot_table`

Analisamos a sobrevivência por sexo e classe.

O valor padrão (default) do argumento aggfunc é a média para o método: [`pandas.DataFrame.pivot_table()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot_table.html)

In [6]:
# Normalmente nos valores vão entrar features quantitativa e nas quebras (linhas e colunas) coloca-se as features categóricas;
data.pivot_table('survived', index = 'sex', columns = 'class')

class,First,Second,Third
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


In [7]:
data.pivot_table('survived', index = 'sex', columns = 'class', aggfunc = 'count')

class,First,Second,Third
sex,,,
female,94,76,144
male,122,108,347


Os argumentos para o método pivot_table são:

* o campo no qual vamos calcular a função de agregação, neste caso `média`
* index define a coluna do DataFrame original cujos valores servem como o índice da tabela dinâmica
* columns define a coluna do DataFrame original cujos valores servem como colunas da tabela dinâmica

Nesta tabela, vemos que mais mulheres do que homens sobreviveram, e quanto maior a classe alta, maior proporção de sobreviventes.

## `pivot_table` multi level

Vamos analisar os grupos de idade como uma terceira dimensão.
 
Para isso, geramos categorias dependendo dos valores da variável 'age' usando `cut`

A função [`pandas.cut()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.cut.html) divide os valores de uma variável em intervalos discretos, com os limites especificados no argumento` bins`.

In [11]:
age_categories = pd.cut(data.age, [0, 18, 36, 80])
data.pivot_table('survived', ['sex', age_categories], 'class')

class               First    Second     Third
sex    age                                   
female (0, 18]   0.909091  1.000000  0.511628
       (18, 36]  0.974359  0.930233  0.511111
       (36, 80]  0.971429  0.823529  0.142857
male   (0, 18]   0.800000  0.600000  0.215686
       (18, 36]  0.527778  0.068966  0.150943
       (36, 80]  0.283333  0.076923  0.069767

In [14]:
age_categories

0      (18.0, 36.0]
1      (36.0, 80.0]
2      (18.0, 36.0]
3      (18.0, 36.0]
4      (18.0, 36.0]
           ...     
886    (18.0, 36.0]
887    (18.0, 36.0]
888             NaN
889    (18.0, 36.0]
890    (18.0, 36.0]
Name: age, Length: 891, dtype: category
Categories (3, interval[int64, right]): [(0, 18] < (18, 36] < (36, 80]]

Vamos adicionar informações sobre as categorias que definem a taxa da variável `fare` de acordo com o quantil ao qual cada valor pertence.

Usamos a função [`pandas.qcut()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.qcut.html) para calcular essas categorias.

Definimos duas categorias de `fare`.

In [15]:
fare_categories = pd.qcut(data.fare, 4)
fare_categories

0       (-0.001, 7.91]
1      (31.0, 512.329]
2       (7.91, 14.454]
3      (31.0, 512.329]
4       (7.91, 14.454]
            ...       
886     (7.91, 14.454]
887     (14.454, 31.0]
888     (14.454, 31.0]
889     (14.454, 31.0]
890     (-0.001, 7.91]
Name: fare, Length: 891, dtype: category
Categories (4, interval[float64, right]): [(-0.001, 7.91] < (7.91, 14.454] < (14.454, 31.0] < (31.0, 512.329]]

In [17]:
data.pivot_table('survived', ['sex', age_categories], [fare_categories, 'class'])

fare            (-0.001, 7.91]           (7.91, 14.454]             
class                    First     Third         Second     Third   
sex    age                                                          
female (0, 18]             NaN  0.625000       1.000000  0.769231  \
       (18, 36]            NaN  0.642857       0.944444  0.352941   
       (36, 80]            NaN  0.000000       0.714286  0.250000   
male   (0, 18]             NaN  0.000000       0.000000  0.461538   
       (18, 36]            0.0  0.129412       0.083333  0.148148   
       (36, 80]            0.0  0.000000       0.133333  0.187500   

fare            (14.454, 31.0]                     (31.0, 512.329]          
class                    First    Second     Third           First Second   
sex    age                                                                  
female (0, 18]             NaN  1.000000  0.400000        0.909091   1.00  \
       (18, 36]       1.000000  0.909091  0.615385        0.972222   1.00   
       (36, 80]       0.800000  0.875000  0.000000        1.000000   1.00   
male   (0, 18]             NaN  0.857143  0.266667        0.800000   0.75   
       (18, 36]       0.818182  0.071429  0.125000        0.416667   0.00   
       (36, 80]       0.333333  0.000000  0.000000        0.277778   0.00   

fare                       
class               Third  
sex    age                 
female (0, 18]   0.142857  
       (18, 36]  0.000000  
       (36, 80]  0.200000  
male   (0, 18]   0.076923  
       (18, 36]  0.750000  
       (36, 80]  0.000000

Podemos especificar diferentes funções de agregação para diferentes colunas do DataFrame original:

In [18]:
data.pivot_table(index = 'sex', 
                 columns = 'class',
                 aggfunc = {'survived':sum, 'fare':'mean'}
                )

fare                       survived             
class        First     Second      Third    First Second Third
sex                                                           
female  106.125798  21.970121  16.118810       91     70    72
male     67.226127  19.741782  12.661633       45     17    47

Para calcular os totais por grupo, usamos `margins` e` margins_name`:

In [19]:
data.pivot_table('survived', 
                 index = 'sex', 
                 columns = 'class', 
                 margins = True, 
                 margins_name = "Total"
                )

class,First,Second,Third,Total
sex,,,,
female,0.968085,0.921053,0.500000,0.742038
male,0.368852,0.157407,0.135447,0.188908
Total,0.629630,0.472826,0.242363,0.383838


---

#### Referências

Python for Data Analysis. Wes McKinney. Cap 10

[`pandas.DataFrame.pivot_table()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot_table.html)